##  CNN 网络结构查看


In [1]:
import numpy as np
import tensorflow as tf

# 设置按需使用GPU
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.InteractiveSession(config=config)

## 2. 构建网络

In [2]:
# 定义卷积层
def conv2d(name, x, filter_shape, strides_x, strides_y, padding):
    """ 
    Args:
        x: 4-D inputs. [batch_size, in_height, in_width, in_channels]
        filter_shape: A list of ints.[filter_height, filter_width, in_channels, out_channels]
        strides: A list of ints. 1-D tensor of length 4. The stride of the sliding window for each dimension of input.
        padding: A string from: "SAME", "VALID". The type of padding algorithm to use.
    Returns:
        h_conv:  A 4-D tensor. [batch_size, out_height, out_width, out_channels]. 
        if padding is 'SAME', then out_height==in_height. 
        else, out_height = in_height - filter_height + 1.
        the same for out_width.
    """
    assert padding in ['SAME', 'VALID']
    strides=[1,strides_x, strides_y,1]
    with tf.variable_scope(name):
        W_conv = tf.get_variable('w', shape=filter_shape)
        b_conv = tf.get_variable('b', shape=[filter_shape[-1]])
        h_conv = tf.nn.conv2d(x, W_conv, strides=strides, padding=padding)
        h_conv_relu = tf.nn.relu(h_conv + b_conv)
    return h_conv_relu
    

def max_pooling(x, k_height, k_width, strides_x, strides_y, padding='SAME'):
    """max pooling layer."""
    ksize=[1,k_height, k_width,1]
    strides=[1,strides_x, strides_y,1]
    h_pool = tf.nn.max_pool(x, ksize, strides, padding)
    return h_pool

def dropout(x, keep_prob, name=None):
    """dropout layer"""
    return tf.nn.dropout(x, keep_prob, name=name)

def fc(name, x, in_size, out_size, activation=None):
    """fully-connect
    Args:
        x: 2-D tensor, [batch_size, in_size]
        in_size: the size of input tensor.
        out_size: the size of output tensor.
        activation: 'relu' or 'sigmoid' or 'tanh'.
    Returns:
        h_fc: 2-D tensor, [batch_size, out_size].
    """
    if activation is not None:
        assert activation in ['relu', 'sigmoid', 'tanh'], 'Wrong activation function.'
    with tf.variable_scope(name):
        w = tf.get_variable('w', shape = [in_size, out_size], dtype=tf.float32)
        b = tf.get_variable('b', [out_size], dtype=tf.float32)
        h_fc = tf.nn.xw_plus_b(x, w, b)
        if activation == 'relu':
            return tf.nn.relu(h_fc)
        elif activation == 'tanh':
            return tf.nn.tanh(h_fc)
        elif activation == 'sigmoid':
            return tf.nn.sigmoid(h_fc)
        else:
            return h_fc

with tf.name_scope('inputs'):
    X_ = tf.placeholder(tf.float32, [None, 784])
    y_ = tf.placeholder(tf.float32, [None, 10])
    X = tf.reshape(X_, [-1, 224, 224, 3])

# # 把X转为卷积所需要的形式
# with tf.variable_scope('cnn_inference'):

In [3]:
# # 自定义的网络
# conv1 = conv2d('conv1', X, [15, 15, 3, 64], 3, 3, 'VALID')
# print(conv1)
# pool1 = max_pooling(conv1, 3, 3, 2, 2, 'VALID')
# print(pool1)

# conv2 = conv2d('conv2', pool1, [5, 5, 64, 128], 1, 1, 'VALID')
# print(conv2)
# pool2 = max_pooling(conv2, 3, 3, 2, 2, 'VALID')
# print(pool2)

# conv3 = conv2d('conv3', pool2, [3, 3, 128, 256], 1, 1, 'SAME')
# print(conv3)
# conv4 = conv2d('conv4', conv3, [3, 3, 256, 256], 1, 1, 'SAME')
# print(conv4)
# conv5 = conv2d('conv5', conv4, [3, 3, 256, 256], 1, 1, 'SAME')
# print(conv5)
# pool5 = max_pooling(conv5, 3, 3, 2, 2, 'VALID')
# print(pool5)

# conv6 = conv2d('conv6', pool5, [6, 6, 256, 512], 1, 1, 'VALID')
# print(conv6)

In [4]:
# alexnet v2
conv1 = conv2d('conv1', X, [11, 11, 3, 64], 4, 4, 'VALID')
print(conv1)
pool1 = max_pooling(conv1, 3, 3, 2, 2, 'VALID')
print(pool1)

conv2 = conv2d('conv2', pool1, [5, 5, 64, 192], 1, 1, 'SAME')
print(conv2)
pool2 = max_pooling(conv2, 3, 3, 2, 2, 'VALID')
print(pool2)

conv3 = conv2d('conv3', pool2, [3, 3, 192, 384], 1, 1, 'SAME')
print(conv3)
conv4 = conv2d('conv4', conv3, [3, 3, 384, 384], 1, 1, 'SAME')
print(conv4)
conv5 = conv2d('conv5', conv4, [3, 3, 384, 256], 1, 1, 'SAME')
print(conv5)
pool5 = max_pooling(conv5, 3, 3, 2, 2, 'VALID')
print(pool5)

Tensor("conv1/Relu:0", shape=(?, 54, 54, 64), dtype=float32)
Tensor("MaxPool:0", shape=(?, 26, 26, 64), dtype=float32)
Tensor("conv2/Relu:0", shape=(?, 26, 26, 192), dtype=float32)
Tensor("MaxPool_1:0", shape=(?, 12, 12, 192), dtype=float32)
Tensor("conv3/Relu:0", shape=(?, 12, 12, 384), dtype=float32)
Tensor("conv4/Relu:0", shape=(?, 12, 12, 384), dtype=float32)
Tensor("conv5/Relu:0", shape=(?, 12, 12, 256), dtype=float32)
Tensor("MaxPool_2:0", shape=(?, 5, 5, 256), dtype=float32)


In [5]:
fc6 = conv2d('fc6', pool5, [5, 5, 256, 4096], 1, 1, 'VALID')
print(fc6)
drop6 = dropout(fc6, 0.5)
print(drop6)
fc7 = conv2d('fc7', drop6, [1, 1, 4096, 4096], 1, 1, 'SAME')
print(fc7)

Tensor("fc6/Relu:0", shape=(?, 1, 1, 4096), dtype=float32)
Tensor("dropout/mul:0", shape=(?, 1, 1, 4096), dtype=float32)
Tensor("fc7/Relu:0", shape=(?, 1, 1, 4096), dtype=float32)
